In [1]:
## Packages lists:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from scipy.optimize import minimize

## Importing all the dataframes from data.ipynb
%run data.ipynb

In [2]:
df_m = monthly_return.pct_change().fillna(0)
df_m.index = pd.to_datetime(df_m.index)

stocktodrop = ['ICT.TUNGGAL PRAKARSA', 'ASTRA AGRO LESTARI',
               'BANK PAN INDONESIA', 'BANK NEGARA INDONESIA',
               'GLOBAL MEDIACOM', 'LIPPO KARAWACI',
               'ASTRA INTERNATIONAL', 'SK HYNIX', 'HYUNDAI ENGR.& CON.']

df_m = df_m.drop(stocktodrop, axis=1)

In [3]:
def roll_min_var_opt(df, start, end):
    stocks = df.columns
    year_weight = end.year + 1
    weights_df = pd.DataFrame(index=stocks)

    for i in range(17):
        ## Resample from start to end
        sample_m = df[df.index.isin(pd.date_range(start, end))]
        tau = len(sample_m)

        ## Compute expected returns
        mu_hat = pd.DataFrame(sample_m.mean(axis=0)).T
    
        ## Excess returns
        excess_returns = sample_m.subtract(mu_hat.values.squeeze(), axis=1)
    
        ## Covariance Matrix
        covmat = 1/tau * excess_returns.T @ excess_returns
    
        # Define objective function (portfolio variance)
        def portfolio_variance(weights, covmat):
            return np.dot(weights.T, np.dot(covmat, weights))
    
        # Define constraint (sum of weights equals 1)
        def constraint(weights):
            return np.sum(weights) - 1
    
        n_assets = len(covmat)
    
        # Initial guess for weights
        initial_weights = np.ones(n_assets) / n_assets
        
        # Define bounds for weights (0 to 1) long-only portfolio
        bounds = [(0, None)] * n_assets
    
        result = minimize(portfolio_variance, initial_weights, args=(covmat,), constraints={'type': 'eq', 'fun': constraint}, bounds=bounds)
    
        optimal_weights = result.x

        weights_df[year_weight] = optimal_weights

        start = dt.datetime(start.year + 1, 1, 1)
        end = dt.datetime(end.year + 1, 1, 1)
        year_weight += 1

    return weights_df

In [4]:
## The minimum variance optimisation worked for 2006 to 2015.
## For 2016 to 2022, it didn't work, so we need to investigate for those years individually.
start = dt.datetime(2000, 1, 1)
end = dt.datetime(2005, 12, 31)
weights = roll_min_var_opt(df_m, start, end)
weights.head()

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
SCHLUMBERGER,8.411694e-18,2.332717e-18,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.390719e-19,2.416190e-20,0.00049,0.00049,0.00049,0.00049,0.00049,0.00049,0.00049
ALUAR,5.530900e-03,6.286662e-18,0.000000e+00,1.684107e-19,1.944785e-18,2.014367e-18,0.000000e+00,3.850123e-18,2.700133e-18,8.647771e-19,0.00049,0.00049,0.00049,0.00049,0.00049,0.00049,0.00049
BBVA BANCO FRANCES,0.000000e+00,1.383917e-19,1.031531e-18,1.315280e-17,2.914805e-18,7.196014e-18,5.584647e-18,0.000000e+00,1.350583e-19,0.000000e+00,0.00049,0.00049,0.00049,0.00049,0.00049,0.00049,0.00049
TERNIUM ARGENTINA SOCIEDAD ANONIMA,1.891143e-05,6.422418e-03,3.122448e-03,0.000000e+00,1.081875e-18,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.300129e-19,0.00049,0.00049,0.00049,0.00049,0.00049,0.00049,0.00049
FLUGHAFEN WIEN,1.997118e-18,0.000000e+00,9.295423e-18,0.000000e+00,1.160377e-18,0.000000e+00,0.000000e+00,2.858556e-18,0.000000e+00,6.765286e-19,0.00049,0.00049,0.00049,0.00049,0.00049,0.00049,0.00049
